In [93]:
import numpy as np
import pandas as pd
from tools.greedy import Greedy
import os

# Comparamos la solución previa con el frente de pareto
 la solución actual con el frente de pareto, ponderamos la dierencia de cada f entre 0 y 1 (cuanto tendría que mejorar d1 para que fuera solución, cuanto d2 para que fuera solución y cuánto d3)

Por ejemplo, solución previa 450, 220, 17. si el 450 fuera 437, sería solución. entonces 1-(437/450). lo mismo con el resto.
La nueva, sacamos lo mismo, 400, 230 y 27. si el 400 fuera 397, sería solución. (397/400)+...

La recompensa será (valor_solucion_actual-valor_solucion_previa)/3. Si se acerca mucho a soluciones, el valor de recompensa será alto.

Si saca una solución exacta del frente de pareto, finalizar y dar un -x?

Si saca una solución nueva, recompensar y dar un +x

In [129]:
####
# F1#------------------------------------------------------------------------------------------
####


def f1(supply_selected, df_distances_demand):
    """
    - df_distances_demand: DataFrame con las distancias entre los puntos de suministro y los puntos de demanda.
    - supply_selected: Lista con los indices de los puntos de suministro seleccionados.
    """

    value = df_distances_demand.iloc[:, supply_selected].min(axis=1).max()
    return value


def max_min_dist(supply_selected, df_distances_demand):
    """
    Calcula la máxima de las mínimas distancias de cada punto de demanda a los puntos de suministro seleccionados.
    """
    return df_distances_demand.iloc[:, supply_selected].min(axis=1).max()


def local_search_f1(solution, value, m, df_distances_demand, n_veces):
    """
    Realiza una búsqueda local para mejorar la solución utilizando el enfoque optimizado de max_min_dist.
    """
    count=0
    count_until_improve=0

    best_solution = solution[:]
    best_objective = value
    len_solution=len(solution)

    while True:
        for i in range(len_solution):
            improved=False
            for j in range(m):
                if j not in solution:
                    temp_solution = solution[:]
                    temp_solution[i] = j
                    temp_objective = max_min_dist(temp_solution, df_distances_demand)

                    if temp_objective < best_objective:
                        best_objective = temp_objective
                        best_solution = temp_solution[:]
                        improved=True
                        count+=1
                        if count==n_veces:
                            return best_solution, best_objective
            if improved:
                count_until_improve=0
            else:
                count_until_improve+=1
                if count_until_improve==len_solution:
                    return best_solution, best_objective



####
# F2#------------------------------------------------------------------------------------------
####


def f2(supply_selected, df_distances_demand):
    """
    - df_distances_demand: DataFrame con las distancias entre los puntos de suministro y los puntos de demanda.
    - supply_selected: Lista con los indices de los puntos de suministro seleccionados.
    """
    asignacion = df_distances_demand.iloc[:, supply_selected].idxmin(axis=1)
    maximum = asignacion.value_counts().max()
    return maximum


def max_demand_per_supply(supply_selected, df_distances_demand):
    """
    Calcula el número máximo de demandas asignadas a un único punto de suministro.
    """
    asignacion = df_distances_demand.iloc[:, supply_selected].idxmin(axis=1)
    return asignacion.value_counts().max()


def local_search_f2(solution, value, m, df_distances_demand, n_veces):
    """
    Realiza una búsqueda local para mejorar la solución minimizando el máximo número de demandas
    asignadas a un único punto de suministro.
    """
    count=0
    count_until_improve=0

    best_solution = solution[:]
    best_objective = value

    len_solution=len(solution)
    
    while True:
        for i in range(len_solution):
            improved=False
            for j in range(m):
                if j not in solution:
                    temp_solution = solution[:]
                    temp_solution[i] = j
                    temp_objective = max_demand_per_supply(
                        temp_solution, df_distances_demand
                    )

                    if temp_objective < best_objective:
                        best_objective = temp_objective
                        best_solution = temp_solution[:]
                        improved=True
                        count+=1
                        if count==n_veces:
                            return best_solution, best_objective
            if improved:
                count_until_improve=0
            else:
                count_until_improve+=1
                if count_until_improve==len_solution:
                    return best_solution, best_objective


####
# F3#------------------------------------------------------------------------------------------
####


def f3(supply_selected, df_distances_demand):
    """
    Calcula la diferencia entre el número máximo y mínimo de demandas asignadas a los puntos de suministro seleccionados,
    de forma más eficiente.
    """
    asignacion = df_distances_demand.iloc[:, supply_selected].idxmin(axis=1)
    counts = asignacion.value_counts()
    return counts.max() - counts.min()


def local_search_f3(solution, value, m, df_distances_demand, n_veces):
    """
    Realiza una búsqueda local para mejorar la solución minimizando la diferencia entre el número
    máximo y mínimo de demandas asignadas a un punto de suministro.
    """
    count=0
    count_until_improve=0

    best_solution = solution[:]
    best_objective = value
    len_solution=len(solution)
    while True:
        for i in range(len_solution):
            improved=False
            for j in range(m):
                if j not in solution:
                    temp_solution = solution[:]
                    temp_solution[i] = j
                    temp_objective = f3(temp_solution, df_distances_demand)

                    if temp_objective < best_objective:
                        best_objective = temp_objective
                        best_solution = temp_solution[:]
                        improved=True
                        count+=1
                        if count==n_veces:
                            return best_solution, best_objective

            if improved:
                count_until_improve=0
            else:
                count_until_improve+=1
                if count_until_improve==len_solution:
                    return best_solution, best_objective



def add_solutions(solution, f1, f2, f3, route_solutions, df_solutions):
    solution = str(sorted(solution))
    solucion_encontrada=False

    if not df_solutions.empty:
        if solution in df_solutions["solution"].values:
            return  # La solución ya existe, no hacer nada

        df_dominado = df_solutions.copy()
        df_dominado = df_dominado[df_dominado["f1"] <= f1]
        df_dominado = df_dominado[df_dominado["f2"] <= f2]
        df_dominado = df_dominado[df_dominado["f3"] <= f3]
        if df_dominado.empty:
            # Eliminar soluciones que sean dominadas por la nueva
            df_solutions = df_solutions[
                ~(
                    (df_solutions["f1"] >= f1)
                    & (df_solutions["f2"] >= f2)
                    & (df_solutions["f3"] >= f3)
                    & (
                        (df_solutions["f1"] > f1)
                        | (df_solutions["f2"] > f2)
                        | (df_solutions["f3"] > f3)
                    )
                )
            ]
            # Agregar la nueva solución
            new_solution = pd.DataFrame(
                [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
            )
            print(new_solution)
            df_solutions = pd.concat([df_solutions, new_solution], ignore_index=True)
            df_solutions.to_csv(route_solutions, index=False)
            solucion_encontrada=True
    else:
        df_solutions = pd.DataFrame(
            [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
        )
        df_solutions.to_csv(route_solutions, index=False)
    return solucion_encontrada


####
# Multi Armed Bandit#------------------------------------------------------------------------------------------
####


def select_arm(context, betha, n_arms, weights):
    if np.random.rand() < betha:  # Exploración: acción aleatoria
        return np.random.randint(0, n_arms)
    else:  # Explotación: mejor acción según el modelo
        # Asegurarse de que el contexto sea un array numpy y tenga la forma correcta
        context_np = np.array(context).reshape(1, -1)  # Convertir a fila vector

        # Calcular la recompensa esperada para cada brazo
        # La recompensa esperada para cada brazo es el producto punto de su vector de pesos y el contexto.
        expected_rewards = np.dot(weights, context_np.T).flatten()

        print(f"Recompensas esperadas para cada brazo: {expected_rewards}")

        exp_rewards = np.exp(expected_rewards - np.max(expected_rewards))
        probabilities = exp_rewards / np.sum(exp_rewards)

        print(f"Probabilidad de coger cada brazo: {probabilities}")

        # Seleccionar un brazo aleatoriamente basado en estas probabilidades
        # np.random.choice permite elegir un elemento de una lista
        # con probabilidades especificadas.
        return np.random.choice(n_arms, p=probabilities)


def decode_action(chosen_arm, parameters=["f1", "f2", "f3"], k_values=[1, 2, 3, 4, 5]):
    param_idx = chosen_arm // len(k_values)
    k_idx = chosen_arm % len(k_values)
    return parameters[param_idx], k_values[k_idx]

def contexto_and_evaluate(f1, f2, f3, df_solutions):
    """
    Calcula la mínima reducción necesaria en f1, f2 o f3 para que la
    combinación no sea dominada por ninguna solución existente en df_solutions.

    Args:
        f1 (float): Valor del primer objetivo.
        f2 (float): Valor del segundo objetivo.
        f3 (float): Valor del tercer objetivo.
        df_solutions (pd.DataFrame): DataFrame con las soluciones existentes.

    Returns:
        tuple: Una tupla (d1, d2, d3) con la reducción  necesaria en cada dimensión.
    """
    # 1. Identificar las soluciones que dominan la combinación actual.
    # Una solución 's' domina a la actual 'c' si s_f1 <= c_f1, s_f2 <= c_f2, Y s_f3 <= c_f3.
    dominating_solutions = df_solutions[
        (df_solutions['f1'] <= f1) &
        (df_solutions['f2'] <= f2) &
        (df_solutions['f3'] <= f3)
    ]

    # 2. Si no hay ninguna solución que la domine, ya es una solución válida.
    # No se necesita ninguna reducción.
    if dominating_solutions.empty:
        return (0, 0, 0)

    # 3. Si hay soluciones dominantes, calcular las diferencias.
    # Estas son las "distancias" que necesitamos superar en cada dimensión.
    delta_f1 = f1 - dominating_solutions['f1']
    delta_f2 = f2 - dominating_solutions['f2']
    delta_f3 = f3 - dominating_solutions['f3']

    # 4. Encontrar la mínima diferencia GLOBAL.
    # Esto representa la reducción más "barata" que podemos hacer para
    # que nuestra combinación deje de ser dominada por al menos una de las soluciones.
    min_delta_f1 = delta_f1.min()
    min_delta_f2 = delta_f2.min()
    min_delta_f3 = delta_f3.min()

    v1=f1/(f1+min_delta_f1)
    v2=f2/(f2+min_delta_f2)
    v3=f3/(f3+min_delta_f3)
    value=(v1+v2+v3)/3

    return (min_delta_f1, min_delta_f2, min_delta_f3), value


def update(chosen_arm, context, reward, weights, route_weights, learning_rate):
    # Asegurarse de que el contexto sea un array numpy
    context_np = np.array(context)

    print(f"Pesos antes: {weights[chosen_arm]}")

    # Actualizar los pesos del brazo elegido.
    # Multiplicamos la recompensa directamente por el contexto y la tasa de aprendizaje.
    # Una recompensa positiva y un contexto dado harán que los pesos se ajusten
    # para favorecer ese brazo en contextos similares.
    # Una recompensa negativa (o baja) hará que los pesos se ajusten en la dirección opuesta,
    # desfavoreciendo ese brazo en contextos similares.
    weights[chosen_arm] += learning_rate * reward * context_np

    print(f"Pesos después: {weights[chosen_arm]}")

    min_val = np.min(weights)
    max_val = np.max(weights)

    # 2. Manejar el caso donde todos los valores son idénticos para evitar división por cero
    if max_val == min_val:
        return np.full(weights.shape, 0)
    
    # 3. Aplicar la fórmula de normalización Min-Max
    reescaled_weights = (weights- min_val) / (max_val - min_val)
    reescaled_weights=np.round(reescaled_weights, 2)
    
    np.save(route_weights, reescaled_weights)

    return


####
# GRASP#------------------------------------------------------------------------------------------
####


def multi_GRASP_Bandit(archive, k, m,context_size, max_iterations=5, alpha=1.0, betha=0.2, i=0, learning_rate=1):

    n_arms=context_size*max_iterations

    folder_distances = "./data/distances/demand/"
    route_distances = folder_distances + archive + ".csv"
    df_distances_demand = pd.read_csv(route_distances)

    folder_solutions = f"Solutions/Multiprocessing/{archive}/"
    # Crea la carpeta si no existe
    os.makedirs(folder_solutions, exist_ok=True)

    route_solutions = folder_solutions + archive + f"_#{i}" + ".csv"

    if os.path.exists(route_solutions):
        df_solutions = pd.read_csv(route_solutions)
    else:
        columnas = ["solution", "f1", "f2", "f3"]
        df_solutions = pd.DataFrame(columns=columnas)

    # Inicializo los pesos para el MAB

    w_dir=f'Weights/{archive}/'
    os.makedirs(w_dir, exist_ok=True)
    route_weights = w_dir + archive + f"_#{i}" + ".npy"
    if os.path.exists(route_weights):
        weights = np.load(route_weights)
    else:
        weights = np.zeros((n_arms, context_size))
        print(f'No había pesos')



    greedy_algorithm = Greedy(df_distances_demand, k, m, alpha)
    """
    Algoritmo GRASP con dos búsquedas locales elegidas aleatoriamente (sin repetición).
    Se mide y muestra el tiempo de ejecución de cada búsqueda local.
    """
    # Etapa Greedy inicial
    solution, f1_value = greedy_algorithm.run()
    print(solution)
    f2_value = f2(solution, df_distances_demand)
    f3_value = f3(solution, df_distances_demand)

    solucion_encontrada=add_solutions(solution, f1_value, f2_value, f3_value, route_solutions, df_solutions)

    if solucion_encontrada:
        print(f'hay nueva solucion: {solution}')
        return solution
    else:
        context, value_prev=contexto_and_evaluate(f1_value, f2_value, f3_value, df_solutions)

    chosen_arm=select_arm(context, betha, n_arms, weights)
    funcion, n_veces = decode_action(chosen_arm, parameters=["f1", "f2", "f3"], k_values=[1, 2, 3, 4, 5])
    print(f'La accion elegida es {funcion} {n_veces} veces')

    # Ejecutar primera búsqueda local
    funcion == "f3"
    if funcion == "f1":
        solution, f1_value = local_search_f1(solution, f1_value, m, df_distances_demand, n_veces)
        f2_value = f2(solution, df_distances_demand)
        f3_value = f3(solution, df_distances_demand)
    elif funcion == "f2":
        solution, f2_value = local_search_f2(solution, f2_value, m, df_distances_demand, n_veces)
        f1_value = f1(solution, df_distances_demand)
        f3_value = f3(solution, df_distances_demand)
    elif funcion == "f3":
        solution, f3_value = local_search_f3(solution, f3_value, m, df_distances_demand, n_veces)
        f1_value = f1(solution, df_distances_demand)
        f2_value = f2(solution, df_distances_demand)

    solucion_encontrada=add_solutions(solution, f1_value, f2_value, f3_value, route_solutions, df_solutions)

    if solucion_encontrada:
        print(f'hay nueva solucion: {solution}')
        reward=5
    else:
        new_context,value_next=contexto_and_evaluate(f1_value, f2_value, f3_value, df_solutions)
        print(f'el valor anterior era {value_prev} y el nuevo es {value_next}')
        reward=(value_next-value_prev)/3
    
    update(chosen_arm, context, reward, weights,route_weights, learning_rate)

    return solution


In [130]:
context_size=3
max_iterations=5

archive="WorkSpace 1000_50_5"
k=5
m= 50
i=0
alpha=0.8
learning_rate=1

In [166]:
multi_GRASP_Bandit(archive, k, m,context_size, max_iterations=max_iterations, alpha=alpha, learning_rate=0.001)

[1, 7, 30, 0, 25]
Recompensas esperadas para cada brazo: [  0.           7.1599518    6.16796144   6.16796144   6.16796144
   0.           0.           0.           0.           0.
   0.55         0.           0.           6.16796144 108.38903593]
Probabilidad de coger cada brazo: [8.45745699e-48 1.08834692e-44 4.03600239e-45 4.03600239e-45
 4.03600239e-45 8.45745699e-48 8.45745699e-48 8.45745699e-48
 8.45745699e-48 8.45745699e-48 1.46589129e-47 8.45745699e-48
 8.45745699e-48 4.03600239e-45 1.00000000e+00]
La accion elegida es f3 5 veces
el valor anterior era 0.8406594493254903 y el nuevo es 0.9365908395106622
Pesos antes: [1.   0.69 0.37]
Pesos después: [1.00202092 0.69115118 0.37175874]


[1, 7, 30, 19, 25]

In [24]:
w_dir=f'Weights/{archive}/'
os.makedirs(w_dir, exist_ok=True)
route_weights = w_dir + archive + f"_#{i}" + ".npy"

In [25]:
weights = np.load(route_weights)

In [26]:
weights

array([[  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [150., 150., 150.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [ 50.,  50.,  50.],
       [ 50.,  50.,  50.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.]])